# Select Tesla Closing Price

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
import sys
!{sys.executable} -m pip install pmdarima

import pmdarima as pm
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Extract TSLA 'Adj Close'
tsla_df = assets['TSLA'].copy()
ts = tsla_df['Adj Close']

# Quick plot
ts.plot(figsize=(10,5), title="Tesla Adjusted Closing Price")
plt.show()


#  Chronological Train-Test Split

In [ ]:
train_size = int(len(ts) * 0.8)
train, test = ts.iloc[:train_size], ts.iloc[train_size:]

print(f"Train size: {len(train)}, Test size: {len(test)}")


#  ARIMA Model

In [ ]:
# Auto-select (p, d, q)
model_auto = pm.auto_arima(train, seasonal=False, trace=True,
                           error_action='ignore', suppress_warnings=True,
                           stepwise=True)
print(model_auto.summary())


In [ ]:
# Fit and Forecast
arima_model = ARIMA(train, order=model_auto.order)
arima_fit = arima_model.fit()

arima_forecast = arima_fit.forecast(steps=len(test))


#  LSTM Model

In [ ]:
# Scale and Generate Sequences
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
test_scaled = scaler.transform(test.values.reshape(-1, 1))

n_input = 30  # lookback window
generator = TimeseriesGenerator(train_scaled, train_scaled, length=n_input, batch_size=32)


# Define & Train
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(n_input, 1)))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit(generator, epochs=20)


# Forecast
pred_list = []
batch = train_scaled[-n_input:].reshape((1, n_input, 1))

for _ in range(len(test)):
    pred = lstm_model.predict(batch, verbose=0)[0]
    pred_list.append(pred)
    batch = np.append(batch[:, 1:, :], [[pred]], axis=1)

lstm_forecast = scaler.inverse_transform(pred_list).flatten()


# Evaluate Models

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

metrics = pd.DataFrame({
    'Model': ['ARIMA', 'LSTM'],
    'MAE': [
        mean_absolute_error(test, arima_forecast),
        mean_absolute_error(test, lstm_forecast)
    ],
    'RMSE': [
        np.sqrt(mean_squared_error(test, arima_forecast)),
        np.sqrt(mean_squared_error(test, lstm_forecast))
    ],
    'MAPE (%)': [
        mape(test, arima_forecast),
        mape(test, lstm_forecast)
    ]
})

print(metrics)


# Plot Forecasts

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test', color='orange')
plt.plot(test.index, arima_forecast, label='ARIMA Forecast', color='green')
plt.plot(test.index, lstm_forecast, label='LSTM Forecast', color='red')
plt.title("Tesla Stock Price Forecast — ARIMA vs LSTM")
plt.legend()
plt.show()
